<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/notebooks/binary_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Neural Networks for Fundamental Binary Functions

This is nice illustration that a neural network can compute anything.


The problem description below is from [The XOR Problem in Neural Networks](https://medium.com/@jayeshbahire/the-xor-problem-in-neural-networks-50006411840b) on medium.com by **Jayesh Bapu Ahire**

Updated by [John Fogarty](https://github.com/jfogarty) for Python 3.6 and [Base2 MLI](https://github.com/base2solutions/mli) and [colab](https://colab.research.google.com) standalone evaluation.

## The XOr Problem

The XOr, or “exclusive or”, problem is a classic problem in ANN research. It is the problem of using a neural network to predict the outputs of XOr logic gates given two binary inputs. An XOr function should return a true value if the two inputs are not equal and a false value if they are equal. All possible inputs and predicted outputs are shown below:

<center><img src="https://github.com/jfogarty/machine-learning-intro-workshop/blob/master/images/xor-function-table.png?raw=1" />
</center>

XOr is a [classification problem](https://en.wikipedia.org/wiki/Statistical_classification) and one for which the expected outputs are known in advance. It is therefore appropriate to use a supervised learning approach.

On the surface, XOr appears to be a very simple problem, however, Minksy and Papert (1969) showed that this was a big problem for neural network architectures of the 1960s, known as perceptrons.

## Perceptrons

Like all ANNs, the perceptron is composed of a network of units, which are analagous to biological neurons. A unit can receive an input from other units. On doing so, it takes the sum of all values received and decides whether it is going to forward a signal on to other units to which it is connected. This is called activation. The [activation function](https://en.wikipedia.org/wiki/Activation_function) uses some means or other to reduce the sum of input values to a 1 or a 0 (or a value very close to a 1 or 0) in order to represent activation or lack thereof. Another form of unit, known as a bias unit, always activates, typically sending a hard coded 1 to all units to which it is connected.

Perceptrons include a single layer of input units — including one bias unit — and a single output unit (see figure 2). Here a bias unit is depicted by a dashed circle, while other units are shown as blue circles. There are two non-bias input units representing the two binary input values for XOr. Any number of input units can be included.

<figure>
  <center><img src="https://github.com/jfogarty/machine-learning-intro-workshop/blob/master/images/simple-perceptron.png?raw=1" />
  <figcaption>Figure 2</figcaption></center>
</figure>



The perceptron is a type of [feed-forward network](http://cs.stanford.edu/people/eroberts/courses/soco/projects/neural-networks/Architecture/feedforward.html), which means the process of generating an output — known as forward propagation — flows in one direction from the input layer to the output layer. There are no connections between units in the input layer. Instead, all units in the input layer are connected directly to the output unit.

A simplified explanation of the forward propagation process is that the input values X1 and X2, along with the bias value of 1, are multiplied by their respective weights W0..W2, and parsed to the output unit. The output unit takes the sum of those values and employs an activation function — typically the [Heavside step function](https://en.wikipedia.org/wiki/Heaviside_step_function) — to convert the resulting value to a 0 or 1, thus classifying the input values as 0 or 1.

It is the setting of the weight variables that gives the network’s author control over the process of converting input values to an output value. It is the weights that determine where the classification line, the line that separates data points into classification groups, is drawn. If all data points on one side of a classification line are assigned the class of 0, all others are classified as 1.

A limitation of this architecture is that it is only capable of separating data points with a single line. This is unfortunate because the XOr inputs are not [linearly separable](https://en.wikipedia.org/wiki/Linear_separability). This is particularly visible if you plot the XOr input values to a graph. As shown in figure 3, there is no way to separate the 1 and 0 predictions with a single classification line.

<figure>
  <center><img src="https://github.com/jfogarty/machine-learning-intro-workshop/blob/master/images/xor_linearly_inseparable.gif?raw=1" />
  <figcaption>Figure 3</figcaption></center>
</figure>


## Multilayer Perceptrons

The solution to this problem is to expand beyond the single-layer architecture by adding an additional layer of units without any direct access to the outside world, known as a hidden layer. This kind of architecture — shown in Figure 4 — is another feed-forward network known as a multilayer perceptron (MLP).

<figure>
  <center><img src="https://github.com/jfogarty/machine-learning-intro-workshop/blob/master/images/xor_multilayer_perceptron.png?raw=1" />
  <figcaption>Figure 4</figcaption></center>
</figure>

It is worth noting that an MLP can have any number of units in its input, hidden and output layers. There can also be any number of hidden layers. The architecture used here is designed specifically for the XOr problem.

Similar to the classic perceptron, forward propagation begins with the input values and bias unit from the input layer being multiplied by their respective weights, however, in this case there is a weight for each combination of input (including the input layer’s bias unit) and hidden unit (excluding the hidden layer’s bias unit). The products of the input layer values and their respective weights are parsed as input to the non-bias units in the hidden layer. Each non-bias hidden unit invokes an activation function — usually the classic [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function) in the case of the XOr problem — to squash the sum of their input values down to a value that falls between 0 and 1 (usually a value very close to either 0 or 1). The outputs of each hidden layer unit, including the bias unit, are then multiplied by another set of respective weights and parsed to an output unit. The output unit also parses the sum of its input values through an activation function — again, the sigmoid function is appropriate here — to return an output value falling between 0 and 1. This is the predicted output.

This architecture, while more complex than that of the classic perceptron network, is capable of achieving non-linear separation. Thus, with the right set of weight values, it can provide the necessary separation to accurately classify the XOr inputs.

<figure>
  <center><img src="https://github.com/jfogarty/machine-learning-intro-workshop/blob/master/images/xor_linear_separable.png?raw=1" />
  <figcaption>Figure 5</figcaption></center>
</figure>

## Backpropagation

The elephant in the room, of course, is how one might come up with a set of weight values that ensure the network produces the expected output. In practice, trying to find an acceptable set of weights for an MLP network manually would be an incredibly laborious task. In fact, it is [NP-complete](https://en.wikipedia.org/wiki/NP-completeness) (Blum and Rivest, 1992). However, it is fortunately possible to learn a good set of weight values automatically through a process known as backpropagation. This was first demonstrated to work well for the XOr problem by Rumelhart et al. (1985).
The backpropagation algorithm begins by comparing the actual value output by the forward propagation process to the expected value and then moves backward through the network, slightly adjusting each of the weights in a direction that reduces the size of the error by a small degree. Both forward and back propagation are re-run thousands of times on each input combination until the network can accurately predict the expected output of the possible inputs using forward propagation.

For the xOr problem, 100% of possible data examples are available to use in the training process. We can therefore expect the trained network to be 100% accurate in its predictions and there is no need to be concerned with issues such as [bias and variance](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff) in the resulting model.

## Conclusion

This note explores the classic ANN XOr problem. The problem itself was described in detail, along with the fact that the inputs for XOr are not linearly separable into their correct classification categories. A non-linear solution — involving an MLP architecture — was explored at a high level, along with the forward propagation algorithm used to generate an output value from the network and the backpropagation algorithm, which is used to train the network.



# A Keras Based Neural Network Solution

**Usage NOTE!** Use `Shift+Enter` to step through this notebook, executing the code as you go.

In [0]:
#@title Welcome
import datetime
print(f"Welcome to exploring this notebook at {datetime.datetime.now()}! ")

In [0]:
class Context:
    VERBOSE=False    # True for extensive logging during execution.
    QUIET=False      # True for minimal logging during execution.
    WARNINGS=False   # True to enable display of annoying but rarely useful messages.

In [0]:
#@title Import Directives
from contextlib import redirect_stderr
import os

import tensorflow as tf

# Suppress Tensorflow log spew.
if Context.WARNINGS:
    import keras    
else:
    tf.logging.set_verbosity(tf.logging.ERROR)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    # Suppress Keras log spew.
    with redirect_stderr(open(os.devnull, "w")):
        import keras
    
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

import numpy as np
import argparse

try:
   device_name = os.environ['COLAB_TPU_ADDR']
   TPU_ADDRESS = 'grpc://' + device_name
   print(f'Running with TPU acceleration at {TPU_ADDRESS}')
except KeyError:
  GPU_NAME = tf.test.gpu_device_name()
  if GPU_NAME.startswith('/device:GPU'): 
      print(f"Running with GPU acceleration at {GPU_NAME}")
  else:
      print("Running on normal CPU without GPU acceleration.")

Running on normal CPU without GPU acceleration.


In [0]:
#@title Text Formatting and Output Functions
def fmt(f, *args):
    if len(args) == 0:
        return str(f)
    else:
        if type(f) is str:
            return f.format(*args)
        else:
            return [str(f)] + [str(s) for s in args]

def log(f, *args):
    print(fmt(f, *args))

def out(f, *args):
    if not Context.QUIET:
        log(fmt(f, *args))

### inputs and training sets for the binary logic functions

In [0]:
Binary_functions = {
    'not' : { 'function' : 'Binary NOT', 'X' : np.array([[0],[1]]),                 
                                         'Y' : np.array([[1],[0]]) },
    
    'xor' : { 'function' : 'Binary XOR', 'X' : np.array([[0,0],[0,1],[1,0],[1,1]]),              
                                         'Y' : np.array([ [0],  [1],  [1],  [0]]) },
    
    'and' : { 'function' : 'Binary AND', 'X' : np.array([[0,0],[0,1],[1,0],[1,1]]),              
                                         'Y' : np.array([ [0],  [0],  [0],  [1]]) },
    
    'or'  : { 'function' : 'Binary OR',  'X' : np.array([[0,0],[0,1],[1,0],[1,1]]), 
                                         'Y' : np.array([ [0],  [1],  [1],  [1]]) }
}

In [0]:
Binary_functions.keys()

dict_keys(['not', 'xor', 'and', 'or'])

### The simple model and training function with no extra confusing stuff

In [0]:
def compute_nofrills(function, X, Y):
    model = Sequential()
    model.add(Dense(8, input_dim=2, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1))
    
    model.fit(X, Y, batch_size=1, epochs=1000, verbose=0)
    model.summary()
    
    y = model.predict(X)
    log("Predicted {} Truth Values from input table:\n{}", function, y)
    log("Exact results:\n{}", Y)       

In [0]:
compute_nofrills(**Binary_functions['xor'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 24        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
Predicted Binary XOR Truth Values from input table:
[[0.00191511]
 [0.99444336]
 [0.99536836]
 [0.00571008]]
Exact results:
[[0]
 [1]
 [1]
 [0]]


### A more complete version with hyperparameters and logging

In [0]:
def compute(function, X, Y, epochs=1000, extralayers=0):
    if Context.VERBOSE:
        out("X.shape={}", X.shape)
        out("Input Array of {} Inputs:\n{}", function, X)
        out("Ouput Array of Truth Results:\n{}".format(Y))

    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation='tanh'))
    for n in range(extralayers):
      model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    log(f"Training {function} model with {extralayers+2} layers in {epochs} iterations")
    model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1))

    out("--------------------------------------------------------------------------")
    out("----- Finding {} Mapping Function using a Brute Force Neural Network:", function)

    model.fit(X, Y, batch_size=1, epochs=epochs, verbose=1 if Context.VERBOSE else 0)

    out("----- Keras information on the function:")
    if not Context.QUIET: model.summary()

    log("--------------------------------------------------------------------------")
    np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})
    y = model.predict(X)
    log("Predicted {} Truth Values from input table:\n{}", function, y)
    log("Exact results:\n{}", Y)
    
    return model

In [0]:
def main():
    for f in Binary_functions.values():
        compute(**f)

In [0]:
#@title Python Command Line Main Program 
try:
    get_ipython
except NameError:
    if __name__=='__main__':
        desc = "This is a brute force method that demonstrates a concept rather than a practical example."
        parser = argparse.ArgumentParser(
            description = "Keras based Neural Network to train for and evaluate Boolean Functions.",
            epilog = desc,
        )    
        parser.add_argument('-v', "--verbose",  help="increase output verbosity.",       action='store_true')
        parser.add_argument('-q', "--quiet",    help="decrease output verbosity.",       action='store_true')
        parser.add_argument('-w', "--warnings", help="don't suppress warning messages.", action='store_true')        
        parser.add_argument('-?',               help=argparse.SUPPRESS,                  action='store_true')

        ap = parser.parse_args()
        if ap.verbose:  Context.VERBOSE=True
        if ap.quiet:    Context.QUIET=True
        if ap.warnings: Context.WARNINGS=True
        if getattr(ap, '?', False):
            parser.print_help()
            exit()
        main()

### Explore the simplest function of all - NOT!

In [0]:
Context.VERBOSE = False
Context.QUIET = True
m = compute('Binary NOT', 
        extralayers=0,
        X=np.array([[0],[1]]),
        Y=np.array([[1],[0]]))

Training Binary NOT model with 2 layers in 1000 iterations
--------------------------------------------------------------------------
Predicted Binary NOT Truth Values from input table:
[[0.99786770]
 [0.00175905]]
Exact results:
[[1]
 [0]]


In [0]:
import random
log("\n----- 0 truthiness:")
for n in [random.randrange(0, 45)/100 for i in range(10)]:
    v = m.predict([[n]])[0][0]
    log(f"- Truthiness of {n:0.6f} is {v:0.6f} [{v <0.5}]")

log("\n----- 1 truthiness:")
for n in [random.randrange(55, 99)/100 for i in range(10)]:
    v = m.predict([[n]])[0][0]
    log(f"- Truthiness of {n:0.6f} is {v:0.6f} [{v <0.5}]")



----- 0 truthiness:
- Truthiness of 0.260000 is 0.961486 [False]
- Truthiness of 0.010000 is 0.997673 [False]
- Truthiness of 0.270000 is 0.955975 [False]
- Truthiness of 0.360000 is 0.852233 [False]
- Truthiness of 0.130000 is 0.992272 [False]
- Truthiness of 0.130000 is 0.992272 [False]
- Truthiness of 0.110000 is 0.993797 [False]
- Truthiness of 0.160000 is 0.989096 [False]
- Truthiness of 0.040000 is 0.996940 [False]
- Truthiness of 0.180000 is 0.986151 [False]

----- 1 truthiness:
- Truthiness of 0.700000 is 0.032847 [True]
- Truthiness of 0.680000 is 0.042363 [True]
- Truthiness of 0.940000 is 0.002747 [True]
- Truthiness of 0.890000 is 0.004191 [True]
- Truthiness of 0.740000 is 0.020104 [True]
- Truthiness of 0.830000 is 0.007429 [True]
- Truthiness of 0.670000 is 0.048197 [True]
- Truthiness of 0.680000 is 0.042363 [True]
- Truthiness of 0.950000 is 0.002539 [True]
- Truthiness of 0.960000 is 0.002351 [True]


### Compute models with various hyperparameters

In [0]:
Context.VERBOSE = False
Context.QUIET = False
compute(**Binary_functions['xor'], extralayers=0, epochs=10)
compute(**Binary_functions['xor'], extralayers=0, epochs=100)
compute(**Binary_functions['xor'], extralayers=0, epochs=1000)
compute(**Binary_functions['xor'], extralayers=1)
compute(**Binary_functions['xor'], extralayers=5)

Training Binary XOR model with 2 layers in 10 iterations
--------------------------------------------------------------------------
----- Finding Binary XOR Mapping Function using a Brute Force Neural Network:
----- Keras information on the function:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8)                 24        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 9         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
--------------------------------------------------------------------------
Predicted Binary XOR Truth Values from input table:
[[0.46597031]
 [0.51728690]
 [0.53256130]
 [0.57734054]]
Exact results:
[[0]
 [1]
 [1]
 [0]]
Training Binary XOR model with 2 layers in 100 iterations
-------------

### End of notebook.